In [124]:
import cv2
import numpy as np

def define_range(path):
    # Original image
    shape = cv2.imread(path,0).shape
    # Range of x_0
    x_0_lower = 0
    x_0_upper = shape[1]*(4/10)
    # Range of x_1
    x_1_lower = shape[1]*(4/10) 
    x_1_upper = shape[1]*(7/10)    

    return x_0_lower, x_0_upper, x_1_lower, x_1_upper

In [125]:
def change_x(coord, x_0_lower, x_0_upper, x_1_lower, x_1_upper):  
    if coord >= x_0_lower and coord <= x_0_upper:
        return 0
    elif coord >= x_1_lower and coord <= x_1_upper:
        return 1
    else:
        return -1

In [126]:
{0: [(47, 'Ant', 'NAMES'), (48, 'SCENA1', 'SCENES')]}

{0: [(47, 'Ant', 'NAMES'), (48, 'SCENA1', 'SCENES')]}

In [134]:
def detect_text(path):
    """
    Detects text in the file.
    Arguments: path (path to local saved image)
    """
    from google.cloud import vision
    import io

    #client = vision.ImageAnnotatorClient.from_service_account_json('/Users/elisamichelet/Documents/EPFLMaster/FDH/Projet/private_key.json')
    client = vision.ImageAnnotatorClient.from_service_account_json('C:/Users/gonxh/Documents/EPFL/Master/MA3/Foundations_of_DH/DH - Rolandi Libretti-a52be57f8f03.json')

    with io.open(path, 'rb') as image_file:
        content = image_file.read()
    
    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    text_result = ""
    dict_bounds = dict()
    
    #Créer ranges de x
    x_0_lower, x_0_upper, x_1_lower, x_1_upper = define_range(path)
    
    #Traiter le texte pour qu'il s'append
    for text in texts:
        text_result += " " + text.description
        coord_x = change_x(text.bounding_poly.vertices[0].x, x_0_lower, x_0_upper, x_1_lower, x_1_upper)
        
        if coord_x != -1:
            #TODO (NAMES, SCENES)
            if coord_x in dict_bounds:
                dict_bounds[coord_x].append((text.bounding_poly.vertices[0].y, text.description))
            else:    
                dict_bounds[coord_x] = [(text.bounding_poly.vertices[0].y, text.description)]

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    return dict_bounds

In [135]:
detect_text('./data/segmented_images/p15_names.png')

{1: [(111,
   'Ant.\nCreon.\nEur.\nCreon\nAnt.\nPiu\nAlc.\nBur.\nCreon,\nEur,\nCreon.\nEur.\nAut.\nCreon.\nll con\n(Sem-\n'),
  (111, 'Ant.'),
  (133, 'Creon.'),
  (158, 'Eur.'),
  (179, 'Creon'),
  (285, 'Ant.'),
  (335, 'Piu'),
  (351, 'Alc.'),
  (373, 'Bur.'),
  (397, 'Creon,'),
  (572, 'Eur,'),
  (593, 'Creon.'),
  (615, 'Eur.'),
  (657, 'Aut.'),
  (747, 'Creon.'),
  (765, 'll'),
  (764, 'con'),
  (785, '(Sem-')]}